# Test of the High-Level API

For development purposes

In [73]:
%load_ext autoreload
%autoreload 2
from __future__ import annotations

import pandas as pd

from tab_err.api import high_level

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
def show_result(original_df: pd.DataFrame, perturbed_df: pd.DataFrame, error_mask: pd.DataFrame | None = None) -> pd.DataFrame:
    """Simple helper function to show DataFrames after perturbing them."""
    return (
        pd.concat([original_df, perturbed_df], keys=["original", "perturbed"], axis=1)
        if error_mask is None
        else pd.concat([original_df, perturbed_df, error_mask], keys=["original", "perturbed", "error_mask"], axis=1)
    )

In [75]:
df_typist_book_title = pd.DataFrame(
    {
        "typist": ["Alice", "Alice", "Alice", "Bob", "Bob", "Bob"],
        "book_title": ["To Kill a Mockingbird", "1984", "Pride and Prejudice", "The Great Gatsby", "Moby-Dick", "The Catcher in the Rye"],
        "rating": [1.0, 3.0, 3.0, 4.0, 2.0, 1.0],
    }
)



print(df_typist_book_title.dtypes)

typist         object
book_title     object
rating        float64
dtype: object


### TEST

In [76]:
df_corrupted, error_mask = high_level.create_errors(df_typist_book_title, 0.5)
show_result(df_typist_book_title, df_corrupted, error_mask)

Column-type dict:  {'typist': [<tab_err.error_type._extraneous.Extraneous object at 0x7f4d9a90ce20>, <tab_err.error_type._mojibake.Mojibake object at 0x7f4d9a90c490>, <tab_err.error_type._replace.Replace object at 0x7f4d9a90f7c0>, <tab_err.error_type._typo.Typo object at 0x7f4d9a90c520>, <tab_err.error_type._missing.MissingValue object at 0x7f4d9a90c0d0>], 'book_title': [<tab_err.error_type._extraneous.Extraneous object at 0x7f4d9a90ce20>, <tab_err.error_type._mojibake.Mojibake object at 0x7f4d9a90c490>, <tab_err.error_type._replace.Replace object at 0x7f4d9a90f7c0>, <tab_err.error_type._typo.Typo object at 0x7f4d9a90c520>, <tab_err.error_type._missing.MissingValue object at 0x7f4d9a90c0d0>], 'rating': [<tab_err.error_type._add_delta.AddDelta object at 0x7f4d9a90d2a0>, <tab_err.error_type._outlier.Outlier object at 0x7f4d9a90de70>, <tab_err.error_type._wrong_unit.WrongUnit object at 0x7f4d9a90f430>, <tab_err.error_type._missing.MissingValue object at 0x7f4d9a90c0d0>]}
Column-mech dict:

/mnt/c/Users/njcha/OneDrive/BHT/Research/tab_err/tab_err/api/high_level.py:130: UserWarning: With an error rate for the column (typist) of: 0.025 and a length of: 6, 0 errors will be introduced.
  col_error_rates = build_column_error_rate_dictionary(data, max_error_rate, col_num_models)
/mnt/c/Users/njcha/OneDrive/BHT/Research/tab_err/tab_err/api/high_level.py:130: UserWarning: With an error rate for the column (book_title) of: 0.025 and a length of: 6, 0 errors will be introduced.
  col_error_rates = build_column_error_rate_dictionary(data, max_error_rate, col_num_models)
/mnt/c/Users/njcha/OneDrive/BHT/Research/tab_err/tab_err/api/high_level.py:130: UserWarning: With an error rate for the column (rating) of: 0.03125 and a length of: 6, 0 errors will be introduced.
  col_error_rates = build_column_error_rate_dictionary(data, max_error_rate, col_num_models)


original                                perturbed                          \
    typist              book_title rating    typist              book_title   
0    Alice   To Kill a Mockingbird    1.0     Alice   To Kill a Mockingbird   
1    Alice                    1984    3.0     Alice                    1984   
2    Alice     Pride and Prejudice    3.0     Alice     Pride and Prejudice   
3      Bob        The Great Gatsby    4.0       Bob        The Great Gatsby   
4      Bob               Moby-Dick    2.0       Bob               Moby-Dick   
5      Bob  The Catcher in the Rye    1.0       Bob  The Catcher in the Rye   

         error_mask                    
  rating     typist book_title rating  
0    1.0      False      False  False  
1    3.0      False      False  False  
2    3.0      False      False  False  
3    4.0      False      False  False  
4    2.0      False      False  False  
5    1.0      False      False  False

# Testing of High Level API with a large dataset

In [77]:
# Read in data
df_clean = pd.read_csv("./clean_beers.csv")
df_clean.head()

,index,id,beer_name,style,ounces,abv,ibu,brewery_id,brewery_name,city,state
0,1,1436,Pub Beer,American Pale Lager,12.0,0.050,NaN,408,10 Barrel Brewing Company,Bend,OR
1,2,2265,Devil's Cup,American Pale Ale (APA),12.0,0.066,NaN,177,18th Street Brewery,Gary,IN
2,3,2264,Rise of the Phoenix,American IPA,12.0,0.071,NaN,177,18th Street Brewery,Gary,IN
3,4,2263,Sinister,American Double / Imperial IPA,12.0,0.090,NaN,177,18th Street Brewery,Gary,IN
4,5,2262,Sex and Candy,American IPA,12.0,0.075,NaN,177,18th Street Brewery,Gary,IN


In [78]:
df_clean = df_clean.drop(columns = ["index", "id", "ounces", "ibu", "brewery_id", "city"])
df_clean.head()

,beer_name,style,abv,brewery_name,state
0,Pub Beer,American Pale Lager,0.050,10 Barrel Brewing Company,OR
1,Devil's Cup,American Pale Ale (APA),0.066,18th Street Brewery,IN
2,Rise of the Phoenix,American IPA,0.071,18th Street Brewery,IN
3,Sinister,American Double / Imperial IPA,0.090,18th Street Brewery,IN
4,Sex and Candy,American IPA,0.075,18th Street Brewery,IN


In [79]:
df_clean["beer_name"] = df_clean["beer_name"].astype(str)
df_clean["style"] = df_clean["style"].astype(str)
df_clean["brewery_name"] = df_clean["brewery_name"].astype(str)
df_clean["state"] = df_clean["state"].astype(str)

In [80]:
df_dirty, error_mask = high_level.create_errors(df_clean, max_error_rate=0.75)
show_result(df_clean, df_dirty, error_mask)

Column-type dict:  {'beer_name': [<tab_err.error_type._extraneous.Extraneous object at 0x7f4d9a90dfc0>, <tab_err.error_type._mojibake.Mojibake object at 0x7f4d9a90cf40>, <tab_err.error_type._replace.Replace object at 0x7f4d9a90e650>, <tab_err.error_type._typo.Typo object at 0x7f4d9a90f820>, <tab_err.error_type._missing.MissingValue object at 0x7f4d9a90c700>], 'style': [<tab_err.error_type._extraneous.Extraneous object at 0x7f4d9a90dfc0>, <tab_err.error_type._mojibake.Mojibake object at 0x7f4d9a90cf40>, <tab_err.error_type._replace.Replace object at 0x7f4d9a90e650>, <tab_err.error_type._typo.Typo object at 0x7f4d9a90f820>, <tab_err.error_type._missing.MissingValue object at 0x7f4d9a90c700>], 'abv': [<tab_err.error_type._add_delta.AddDelta object at 0x7f4d9a90e5c0>, <tab_err.error_type._outlier.Outlier object at 0x7f4d9a90dc90>, <tab_err.error_type._wrong_unit.WrongUnit object at 0x7f4d9a90df30>, <tab_err.error_type._missing.MissingValue object at 0x7f4d9a90c700>], 'brewery_name': [<tab_

original                                         \
                 beer_name                           style    abv   
0                 Pub Beer             American Pale Lager  0.050   
1              Devil's Cup         American Pale Ale (APA)  0.066   
2      Rise of the Phoenix                    American IPA  0.071   
3                 Sinister  American Double / Imperial IPA  0.090   
4            Sex and Candy                    American IPA  0.075   
...                    ...                             ...    ...   
2405             Belgorado                     Belgian IPA  0.067   
2406         Rail Yard Ale        American Amber / Red Ale  0.052   
2407       B3K Black Lager                     Schwarzbier  0.055   
2408   Silverback Pale Ale         American Pale Ale (APA)  0.055   
2409  Rail Yard Ale (2009)        American Amber / Red Ale  0.052   

                                                  perturbed  \
                   brewery_name state             beer_name   
0     10 Barrel Brewing Company    OR              Pub Beer   
1           18th Street Brewery    IN                  <NA>   
2           18th Street Brewery    IN                  <NA>   
3           18th Street Brewery    IN              Sinister   
4           18th Street Brewery    IN         Sex and Candy   
...                         ...   ...                   ...   
2405    Wynkoop Brewing Company    CO             Belgorado   
2406    Wynkoop Brewing Company    CO         Rail Yard Ale   
2407    Wynkoop Brewing Company    CO       B3K Black Lager   
2408    Wynkoop Brewing Company    CO   Silverback Pale Ale   
2409    Wynkoop Brewing Company    CO  Rail Yard Ale (2009)   

                                                                              \
                               style    abv               brewery_name state   
0                American Pale Lagrr  0.050  10 Barrel Brewing Company    OR   
1            American Pale Ale (APA)  0.660        18th Street Brewery    IJ   
2                       American IPA  0.071        18th Street Brewerz    IN   
3     American Double / Im-erial IPA  0.090        18th Street Breaery    IN   
4                       A,erican IPA  0.075        18th Street Brewerz    IN   
...                              ...    ...                        ...   ...   
2405                     Belgoan IPA  0.670    Wynkoop Brewing Company    CO   
2406       .American Amber / Red Ale  0.052    Wynkoop Brewing Cojpany    CO   
2407                     Schwarzbier  0.055    Wznkoop Brewing Companz    CO   
2408        .American Pale Ale (APA)  0.055    Wznkoop Brewing Companz    CP   
2409        American Amber / Red Ale  0.052    Wynkoop Brewing Company  <NA>   

     error_mask                                    
      beer_name  style    abv brewery_name  state  
0          True   True  False        False   True  
1          True   True   True        False   True  
2          True  False  False         True   True  
3          True   True  False         True  False  
4          True   True  False         True  False  
...         ...    ...    ...          ...    ...  
2405       True   True   True         True  False  
2406       True   True  False         True   True  
2407      False  False  False         True  False  
2408       True   True  False         True   True  
2409       True   True  False         True   True  

[2410 rows x 15 columns]

In [81]:
error_mask.mean(axis=None)
error_mask.mean(axis=0)

beer_name       0.746888
style           0.746888
abv             0.746888
brewery_name    0.746888
state           0.746888
dtype: float64